# Script 2 - Calculate, plot, and save monthly climatology and test to divide in subregions and select candidates for "cut off" SST

Regional Areas:

| EBUS | Subregion | Area | 
| --- | --- | ---| 
| Benguela | Poleward | 194997 | 
| Benguela | Central | 197744 | 
| Benguela | Equatorward | 198233 | 
| California | Poleward | 207197 | 
| California | Central | 196898 | 
| California | Equatorward | 190667 |
| Humboldt | Poleward | 462219 | 
| Humboldt | Central | 324025 | 
| Humboldt | Equatorward | 192436 | 
| Iberian/Canary | Poleward |  | 
| Iberian/Canary | Central | 255325 | 
| Iberian/Canary | Equatorward | 179848 | 

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import datetime as dt
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import math
import warnings 
warnings.simplefilter('ignore') 
import seaborn as sns
import glob

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from calendar import month_abbr
from geopy import distance

from matplotlib.ticker import StrMethodFormatter
#from matplotlib_scalebar.scalebar import ScaleBar
from datetime import datetime
import matplotlib.dates as mdates
from scalebar import scale_bar
from datetime import datetime


ddir = '../data/data_download/'
ebus = 'Iberian-Canary' #'California' #'Humboldt' #'Bengula'


## Calculate Climatology

In [ ]:
ddir+ebus+'/*.nc'

In [ ]:
#read in all the data files from Google Drive
ddir = '../data/data_download/'
ebus = 'Iberian-Canary' #'Humboldt' #'California' #'Bengula'
fns = glob.glob(ddir+ebus+'/*.nc')
ds = xr.open_mfdataset(fns)
ds

In [ ]:
#calculate the average for each month and save output
clim = ds.groupby('time.month').mean('time',keep_attrs=True)
clim.to_netcdf(ddir+'/'+ebus+'_monthlyClim.nc')

In [ ]:
#calculate the average across all time
clim = ds.mean('time',keep_attrs=True)
clim.to_netcdf(ddir+'/'+ebus+'_TotalClim.nc')

In [ ]:
###Calculate the nearshore and offshore bands for 30km
#select either nearshore or offshore
onshore_offshore = 'Offshore' #'Nearshore'

#calculate nearshore bands
#create empty xarray dataset
nearshore_df = pd.DataFrame()

#loop through every latitute
for i in range(len(ds.lat)):
    #select out just the latitude
    clim_temp = ds.isel(lat = i)
    #filter out the NAs
    clim_temp = clim_temp.analysed_sst.where(~np.isnan(clim_temp), drop = True)
    
    if onshore_offshore == 'Nearshore':
        #select the last 30 km (represents the closest nearshore longitudes)
        clim_temp = clim_temp.isel(lon = slice(-31, -1))
    else: 
        #select the first 30 km (represents the furthest offshore longitudes)
        clim_temp = clim_temp.isel(lon = slice(0, 30))
        
    #convert to pandas
    clim_temp = clim_temp.to_dataframe().reset_index().round(3)
    
    #if the value is zero (ie if there aren't any values)
    if len(clim_temp.time) == 0:
        #skip that latitude
        pass
    
    #otherwise 
    else:   
        #take the mean
        #clim_temp = clim_temp.groupby(['time', 'lat']).mean('analysed_sst').drop('lon', axis = 1)    

        #join data
        nearshore_df = pd.concat([nearshore_df, clim_temp])
                
nearshore_df.set_index(['time', 'lon', 'lat']).to_xarray().to_netcdf(ddir+'/Offshore_Onshorebands/'+ebus+'_'+onshore_offshore+'MonthlyClim_30km.nc')
print('Finished ' + ebus + ' ' + onshore_offshore)

## Calculate the SST Threshold for Subregions using the Nearshore Data

In [ ]:
ebus = 'Iberian-Canary'
test = xr.open_dataset('../data/data_download/Iberian-Canary_monthlyClim.nc')
test.close()

#select out coordinates based on EBUS
if ebus == 'California':
    lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
    lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
elif ebus == 'Humboldt':
    lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
    lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
elif ebus == 'Iberian-Canary':
    #old boundary
    # lats = np.array([[15,21.33],[21.33,30],[37, 43.39]]) 
    # lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
    #new boundary (shift from 30 to 33°N)
    lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
    lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 

else: #Benguela
    lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
    lons = np.array([[13, 20],[10, 17],[8, 15]])


for i in range(3):
    print('Region' + str(i) + ': ' + str(np.round(float(test.analysed_sst.sel(lat = slice(lats[i][0], lats[i][1])).mean(...).values), 2)))

In [ ]:
ebus = 'Iberian-Canary'

clim = xr.open_dataset(ddir+'/Offshore_Onshorebands/'+ebus+'_NearshoreMonthlyMean_30km.nc')
clim.close()

#select out coordinates based on EBUS
if ebus == 'California':
    lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
    lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
elif ebus == 'Humboldt':
    lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
    lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
elif ebus == 'Iberian-Canary':
    lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
    lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
else: #Benguela
    lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
    lons = np.array([[13, 20],[10, 17],[8, 15]])
                
# cutoff SST (median)
#sst_co = clim0.analysed_sst.median()
#sst_co

# cutoff SST (mean)
sst_co_list = []
[sst_co_list.append(float(clim.analysed_sst.sel(lat = slice(lats[i][0], lats[i][1])).mean())) for i in range(len(lats))]
sst_co_list
#sst_co = clim.analysed_sst.mean()
#sst_co.values

# Regional Figures

In [ ]:
#read in new climatology file
cali = xr.open_dataset(ddir+'California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()


beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()


iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 20],[10, 17],[8, 15]])

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(cali.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(cali.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(cali.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(cali.analysed_sst.lon)])

latr = [min_lat, max_lat]
lonr = [min_lon, max_lon]

#create a region variable with a margin
margin = 1
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#add state outlines
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree())

#ax.add_feature(states_provinces, linewidth = 0.5)

ax.coastlines(resolution='50m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'black')
ax.set_xticks([*np.arange(-180,180,5)], crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region_raw[0,0])+1,region_raw[0,1]+1,6)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

cali.analysed_sst.plot(cmap = 'jet', add_colorbar=False, vmin = 11, vmax = 26)

for ds_lats, ds_lons in zip(lats_combined, lons_combined):
    for i in range(len(ds_lats)):
        rect2 = patches.Rectangle((ds_lons[i][0], ds_lats[i][0]), (ds_lons[i][1]-ds_lons[i][0]), (ds_lats[i][1]-ds_lats[i][0]), linewidth=1, edgecolor='k', linestyle='--',facecolor='none')
        ax.add_patch(rect2)
        
plt.text(cali_lons[0][1]+0.2, cali_lats[0][1]+0.2, "Equatorward", ha='right', fontsize = 16)
plt.text(cali_lons[1][1]+0.2, cali_lats[1][1]-0.7, "Central", ha='left', fontsize = 16)
plt.text(cali_lons[2][1]+0.2, cali_lats[2][1]-0.7, "Poleward", ha='left', fontsize = 16)

#add scalebar
scale_bar(ax, (0.1, 0.06), 2_50)

#add north arrow
x, y, arrow_length = 0.11, 0.18, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)


plt.xlabel('Longitude')
plt.ylabel('Latitude')
#plt.title('California Average Sea Surface Temperature (2002-2022)', fontsize = 16)
plt.title('California Upwelling Regions', fontsize = 16)
plt.tight_layout()
plt.savefig('../figures/California_Subregions_annual.png', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()

In [ ]:
#read in new climatology file
cali = xr.open_dataset(ddir+'California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()


beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()


iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 20],[10, 17],[8, 15]])

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(hum.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(hum.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(hum.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(hum.analysed_sst.lon)])

latr = [min_lat, max_lat]
lonr = [min_lon-2, max_lon]

#create a region variable with a margin
margin = 1
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='50m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'black')
ax.set_xticks([*np.arange(-180,180,5)], crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0]),region[0,1]+1,10)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

hum.analysed_sst.plot(cmap = 'jet', add_colorbar=False, vmin = 11, vmax = 26)

for ds_lats, ds_lons in zip(lats_combined, lons_combined):
    for i in range(len(ds_lats)):
        rect2 = patches.Rectangle((ds_lons[i][0], ds_lats[i][0]), (ds_lons[i][1]-ds_lons[i][0]), (ds_lats[i][1]-ds_lats[i][0]), linewidth=1, edgecolor='k', linestyle='--',facecolor='none')
        ax.add_patch(rect2)
        
plt.text(hum_lons[0][0]-0.2, hum_lats[0][1]-0.9, "Poleward", ha='right', fontsize = 16)
plt.text(hum_lons[1][0]-0.2, hum_lats[1][1]-1.1, "Central", ha='right', fontsize = 16)
plt.text(hum_lons[2][0]+2.1, hum_lats[2][1]+0.4, "Equatorward", ha='right', fontsize = 16)

#add scalebar
scale_bar(ax, (0.1, 0.04), 2_50)

#add north arrow
x, y, arrow_length = 0.11, 0.17, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Humboldt Upwelling Regions', fontsize = 16)
plt.tight_layout()
plt.savefig('../figures/Humboldt_Subregions_annual.png', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()


In [ ]:
#read in new climatology file
cali = xr.open_dataset(ddir+'California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()


beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()


iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 20],[10, 17],[8, 15]])

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(beng.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(beng.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(beng.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(beng.analysed_sst.lon)])

latr = [min_lat, max_lat]
lonr = [min_lon, max_lon]

#create a region variable with a margin
margin = 1
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='50m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'black')
ax.set_xticks(np.round([*np.arange(region_raw[1,0],region_raw[1,1]+1,5)],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0])+3,region[0,1]+1,10)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

fg = beng.analysed_sst.plot(cmap = 'jet', vmin = 11, vmax = 26, add_colorbar=False)
cb = plt.colorbar(fg, pad = 0.02)      
cb.set_label(label = 'Sea Surface Temperature (SST, °C)', size = 16, labelpad = 10)


for ds_lats, ds_lons in zip(lats_combined, lons_combined):
    for i in range(len(ds_lats)):
        rect2 = patches.Rectangle((ds_lons[i][0], ds_lats[i][0]), (ds_lons[i][1]-ds_lons[i][0]), (ds_lats[i][1]-ds_lats[i][0]), linewidth=1, edgecolor='k', linestyle='--',facecolor='none')
        ax.add_patch(rect2)
        
plt.text(beng_lons[0][0]-0.2, beng_lats[0][1]-0.9, "Poleward", ha='right', fontsize = 16)
plt.text(beng_lons[1][0]-0.2, beng_lats[1][1]-0.9, "Central", ha='right', fontsize = 16)
plt.text(beng_lons[2][0]+4, beng_lats[2][1]+0.3, "Equatorward", ha='right', fontsize = 16)

#add scalebar
scale_bar(ax, (0.1, 0.04), 2_50)

#add north arrow
x, y, arrow_length = 0.119, 0.165, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Benguela Upwelling Regions', fontsize = 16)
plt.tight_layout()
plt.savefig('../figures/Benguela_Subregions_annual.png', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()


In [ ]:
#read in new climatology file
cali = xr.open_dataset(ddir+'California_TotalClim.nc')
cali.close()

hum = xr.open_dataset(ddir+'Humboldt_TotalClim.nc')
hum.close()


beng = xr.open_dataset(ddir+'Bengula_TotalClim.nc')
beng.close()


iber = xr.open_dataset(ddir+'Iberian-Canary_TotalClim.nc')
iber.close()

#select out coordinates based on EBUS
cali_lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
cali_lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
hum_lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
hum_lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
iber_lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
iber_lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
beng_lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
beng_lons = np.array([[13, 20],[10, 17],[8, 15]])

#extract the minimum and maximum latitudes and longitudes from all the masks
max_lat = np.nanmax([np.nanmax(iber.analysed_sst.lat)])
min_lat = np.nanmin([np.nanmin(iber.analysed_sst.lat)])

max_lon = np.nanmax([np.nanmax(iber.analysed_sst.lon)])
min_lon = np.nanmin([np.nanmin(iber.analysed_sst.lon)])

latr = [min_lat, max_lat]
lonr = [min_lon, max_lon]

#create a region variable with a margin
margin = 1
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 
region_raw = np.array([[latr[0],latr[1]],[lonr[0],lonr[1]]]) 

#create and set the figure context
fig = plt.figure(figsize=(16,10), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='50m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(cfeature.BORDERS, color = 'black')
ax.set_xticks(np.round([*np.arange(region_raw[1,0]-2,region_raw[1,1]+1,6)],0), crs=ccrs.PlateCarree()) 
ax.set_yticks(np.round([*np.arange(np.floor(region[0,0])+3,region[0,1]+1,10)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
ax.gridlines(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')

lats_combined = [cali_lats, hum_lats, iber_lats, beng_lats]
lons_combined = [cali_lons, hum_lons, iber_lons, beng_lons]

iber.analysed_sst.plot(cmap = 'jet', add_colorbar=False, vmin = 11, vmax = 26)

for ds_lats, ds_lons in zip(lats_combined, lons_combined):
    for i in range(len(ds_lats)):
        rect2 = patches.Rectangle((ds_lons[i][0], ds_lats[i][0]), (ds_lons[i][1]-ds_lons[i][0]), (ds_lats[i][1]-ds_lats[i][0]), linewidth=1, edgecolor='k', linestyle='--',facecolor='none')
        ax.add_patch(rect2)
        
plt.text(iber_lons[0][0]+3.5, iber_lats[0][1]-7.4, "Equatorward", ha='right', fontsize = 16)
plt.text(iber_lons[1][0]-0.2, iber_lats[1][1]-0.9, "Central", ha='right', fontsize = 16)
plt.text(iber_lons[2][0]-0.2, iber_lats[2][1]-0.9, "Poleward", ha='right', fontsize = 16)

#add scalebar
scale_bar(ax, (0.1, 0.93), 2_50)

#add north arrow
x, y, arrow_length = 0.115, 0.905, 0.075
ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=2, headwidth=10),
            ha='center', va='center', fontsize=16,
            xycoords=ax.transAxes)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Iberian-Canary Upwelling Regions', fontsize = 16)
plt.tight_layout()
plt.savefig('../figures/Iberian-Canary_Subregions_annual.png', facecolor='white', bbox_inches = 'tight', dpi = 300)
plt.show()
